In [ ]:
import sqlite3
import pandas as pd

In [ ]:
con = sqlite3.connect(':memory:')

In [ ]:
table_names = [
    'Categories',   # категории товаров
    'Customers',    # заказчики
    'Employees',    # сотрудники
    'OrderDetails', # детали заказов
    'Orders',       # заказы
    'Products',     # товары
    'Shippers',     # перевозчики
    'Suppliers',    # поставщики
]

In [ ]:
for table in table_names:
    data = pd.read_csv(f'{table}.csv', sep = ';')
    data.to_sql(table, con, index=False, if_exists='replace')

In [ ]:
pd.read_sql(
    '''
    SELECT * FROM Categories
    ''',
    con,
)

In [ ]:
tasks = 0

### Все задания нужно выполнить только с помощью sql

1. Выберите заказчиков из Германии, Франции и Мадрида, выведите их название, страну и адрес.

In [ ]:
tasks+=1
pd.read_sql(
    '''
    SELECT * FROM Categories
    ''',
    con,
)

2. Выберите топ 3 страны по количеству заказчиков, выведите их названия и количество записей.

In [ ]:
tasks+=1
pd.read_sql(
    '''
    SELECT * FROM Categories
    ''',
    con,
)

3. Выберите перевозчика, который отправил 10-й по времени заказ, выведите его название, и дату отправления.

In [ ]:
tasks+=1
pd.read_sql(
    '''
    SELECT * FROM Categories
    ''',
    con,
)

4. Выберите самый дорогой заказ, выведите список товаров с их ценами.

In [ ]:
tasks+=1
pd.read_sql(
    '''
    SELECT * FROM Categories
    ''',
    con,
)

5. Какой товар больше всего заказывали по количеству единиц товара, выведите его название и количество единиц в каждом из заказов.

In [ ]:
tasks+=1
pd.read_sql(
    '''
    SELECT * FROM Categories
    ''',
    con,
)

6. Выведите топ 5 поставщиков по количеству заказов, выведите их названия, страну, контактное лицо и телефон.

In [ ]:
tasks+=1
pd.read_sql(
    '''
    SELECT * FROM Categories
    ''',
    con,
)

7. Какую категорию товаров заказывали больше всего по стоимости в Бразилии, выведите страну, название категории и сумму.

In [ ]:
tasks+=1
pd.read_sql(
    '''
    SELECT * FROM Categories
    ''',
    con,
)

8. Какая разница в стоимости между самым дорогим и самым дешевым заказом из США.

In [ ]:
tasks+=1
pd.read_sql(
    '''
    SELECT * FROM Categories
    ''',
    con,
)

9. Выведите количество заказов у каждого их трех самых молодых сотрудников, а также имя и фамилию во второй колонке.

In [ ]:
tasks+=1
pd.read_sql(
    '''
    SELECT * FROM Categories
    ''',
    con,
)

10. Сколько банок крабового мяса всего было заказано.

In [ ]:
tasks+=1
pd.read_sql(
    '''
    SELECT * FROM Categories
    ''',
    con,
)

In [ ]:
if tasks==10:
    print('Выполнены все задания 🙂')
else:
    print('Выполнены не все задания! 🙀')